In [29]:
import torch
from PoetryDataset import word2ix, ix2word, PoetryDataset
from config import Config
from Model.LSTM import Model

poetry_dataset = PoetryDataset()
cfg = Config()

In [30]:
# 加载模型
# from Model.pretrained_LSTM import Model
model = Model()
model.load_state_dict(torch.load('LSTM_3.pth'))
# model.load_state_dict(torch.load('BiLSTM.pth'))
model.cuda()
model.eval()

Model(
  (embeddings): Embedding(8293, 100)
  (lstm): LSTM(100, 1024, num_layers=3, batch_first=True)
  (output): Sequential(
    (0): Linear(in_features=1024, out_features=2048, bias=True)
    (1): Tanh()
    (2): Dropout(p=0.3, inplace=False)
    (3): Linear(in_features=2048, out_features=4096, bias=True)
    (4): Tanh()
    (5): Dropout(p=0.3, inplace=False)
    (6): Linear(in_features=4096, out_features=8293, bias=True)
  )
)

In [31]:
def write_poetry_from_first_sentence(first_sentence):
    poetry = list(first_sentence)
    # 初始隐状态
    bidirectional_num = 2 if cfg.bidirectional else 1
    hidden = torch.zeros((2, cfg.lstm_layers*1, 1, cfg.hidden_size), dtype=torch.float).cuda()
    input = torch.Tensor([word2ix['<START>']]).view(1,1).long().cuda()
    with torch.no_grad():
        for i in range(cfg.seq_len):
            output, hidden = model(input, hidden)
            word_predicted = ''
            # 读取第一句
            if i < len(first_sentence):
                word_predicted = first_sentence[i]
                input = input.data.new([word2ix[word_predicted]]).view(1,1)
            else:
                top_idx = output.data[0].topk(1)[1][0].item()
                word_predicted = ix2word[top_idx]
                poetry.append(word_predicted)
                input = input.data.new([top_idx]).view(1,1)
            
            if word_predicted == '<START>':
                continue
            if word_predicted == '<EOP>' or word_predicted == '<EOP>':
                poetry.pop(-1)
                if len(poetry) < 23:
                    continue
                break
    return ''.join(poetry)
    

def write_poetry_from_first_chars(firstchars):
    poetry = []
    count = 0
    bidirectional_num = 2 if cfg.bidirectional else 1
    hidden = torch.zeros((2, cfg.lstm_layers*1, 1, cfg.hidden_size), dtype = torch.float).cuda()
    input = torch.Tensor([word2ix['<START>']]).view(1,1).long().cuda()
    with torch.no_grad():
        for i in range(cfg.seq_len):
            output, hidden = model(input, hidden)
            word_predicted = ''
            if i == 0 or (poetry[-1] in ['，', '。'] and count < len(firstchars)):
                word_predicted = firstchars[count]
                count += 1
                input = input.data.new([word2ix[word_predicted]]).view(1,1)
            else:
                top_idx = output.data[0].topk(1)[1][0].item()
                word_predicted = ix2word[top_idx]
                input = input.data.new([top_idx]).view(1, 1)
            
            if word_predicted == '<START>':
                continue
            poetry.append(word_predicted)
            if word_predicted == '<EOP>' or word_predicted == '<EOP>':
                poetry.pop(-1)
                if len(poetry) < 23:
                    continue
                break
    return ''.join(poetry)
            

In [32]:
print(write_poetry_from_first_sentence('人闲桂花落'))
print(write_poetry_from_first_sentence('驿外断桥边'))
print(write_poetry_from_first_sentence('赵客缦胡缨'))
print(write_poetry_from_first_sentence('明月几时有'))
print(write_poetry_from_first_sentence('春江潮水连海平'))
print(write_poetry_from_first_sentence('天门中断楚江开'))
print(write_poetry_from_first_sentence('昨夜星辰昨夜风'))
print(write_poetry_from_first_sentence('朝辞白帝彩云间'))
print(write_poetry_from_first_sentence('葡萄美酒夜光杯'))
print(write_poetry_from_first_sentence('东风夜放花千树'))
print(write_poetry_from_first_sentence('紫泉宫殿锁烟霞'))
print(write_poetry_from_first_sentence('花近高楼伤客心'))
print(write_poetry_from_first_sentence('积雨空林烟火迟'))
print(write_poetry_from_first_sentence('身向云山那畔行'))
print(write_poetry_from_first_sentence('落魄江湖载酒行'))

人闲桂花落，夜静竹声起。秋夜不可望，空堂空寂寞。
驿外断桥边，残灯落上边。客愁当夜直，山信有灯眠。
赵客缦胡缨，王孙不可寻。水边三月尽，江上几人归。云去山中路，云归江上楼。谁知故山远，江上有云泥。
明月几时有，此心空自伤。不知山上月，何处月明明。
春江潮水连海平，江南江北长路岐。楚歌不复采苹歌，回首青山望故乡。
天门中断楚江开，水国秋来日日回。江上日高三尺劒，不知何处是长云。
昨夜星辰昨夜风，月明花月满宫尘。今朝不是当时事，不似君家一尺茵。
朝辞白帝彩云间，今日相逢泪满巾。江上路遥云不信，洞庭云外日初曛。江南客梦应如梦，不觉离情似去年。
葡萄美酒夜光杯，夜击倾壺泪满巾。不是人间无事态，不妨相向向东来。
东风夜放花千树，远水初融日上面。柳絮柳垂花，花落莺啼水风急。玉钗垂枕柳烟前，酒后管弦人半展。枕上回风屏不稳，枕上容华枕边望。
紫泉宫殿锁烟霞，不是人间不肯听。今日相逢花更发，不堪回首望长安。
花近高楼伤客心，月明高柳影萧萧。不堪回首思王子，夜夜思君泪满衣。
积雨空林烟火迟，孤舟一夜隔江樯。江南江北多別地，日日江南一客书。
身向云山那畔行，人间道路不堪论。何如今日从公事，不得相如为老郎。
落魄江湖载酒行，一声渔父醉离情。江南江北多离別，一夜江山一望归。


In [33]:
print(write_poetry_from_first_chars('朱健行'))
print(write_poetry_from_first_chars('戢启瑞'))
print(write_poetry_from_first_chars('球王牛逼'))
print(write_poetry_from_first_chars('阎熠光'))
print(write_poetry_from_first_chars('祝捷'))
print(write_poetry_from_first_chars('祝平'))
print(write_poetry_from_first_chars('胡小泉'))
print(write_poetry_from_first_chars('胡昌泰'))
print(write_poetry_from_first_chars('胡晓泉'))
print(write_poetry_from_first_chars('深度学习'))
print(write_poetry_from_first_chars('中国科学院大学'))
print(write_poetry_from_first_chars('我爱自然语言处理'))

朱门道士无主人，健笔劳王事日回。行到西京多少事，一生长安是长尘。
戢板三年不是平，启门关阙在云涯。瑞花春尽春云白，不是江南一片花。
球上青松子，王孙白日看。牛羊归客路，逼谷旧家山。
阎浮云外寺，熠爚出西秦。光彩生三径，星光入四邻。云开千仞阁，水喷一川龙。石窦生香竹，巖根带落虹。何当一回首，更向洞庭看。
祝融南去，捷客开门。云生上巳，风度东门。云开三径，水落五湖。
祝融南去，平津旧辇前。有山非石室，无处不莓苔。石室人来去，丹楹水似鸿。何当从此会，一为奏清尘。
胡尘去国去，小苑更春来。泉合冰光出，宫深月影空。愿当天上月，长与路傍同。
胡尘去国去，昌路塞鸿休。泰末归来去，空余老去心。
胡尘去国去，晓月单衣单。泉咽鸣不定，天边生暮寒。
深山有巢路，度世复多尘。学道人间事，习家道在东。
中峰高树一枝花，国向南天尽日流。科斗只应知有意，学山长自足为禽。院中春日花如雪，大道东山是是谁。学道不成何处去，老僧相伴老松堂。
我有一心宝，爱得三十年。自非不得利，然后不由人。语者与我同，言是无为人。处处生尘埃，理生无定準。若能如是达，不是不知死。不是不生
